In [34]:
import pandas as pd

google = pd.read_csv("./google_ads.csv")
meta = pd.read_csv("./meta_ads.csv")
email= pd.read_csv("./email_marketing.csv")

In [35]:
print("Google ads:")
print(google.head())
print("Meta ads:")
print(meta.head())
print("Email ads:")
print(email.head())

Google ads:
                            campaign_id campaign_name        date  \
0  16a0a598-498c-4360-acbd-73fd5de3c96c    Campaña 10  2024-02-06   
1  095727ce-b041-47b8-b4a3-5bdc81e9fb10    Campaña 10  2024-02-07   
2  3aa3a3d5-b342-4fe1-b6c4-8e69f0924bef     Campaña 8  2024-02-23   
3  d05b281b-deb2-4e0d-bd4e-b0fda784ad72     Campaña 6  2024-02-14   
4  06f8ef35-9939-45bf-bc16-d02d1074940e     Campaña 1  2024-04-20   

   impressions  clicks  cost_usd  
0         8270     485       960  
1         6390     156       171  
2         6734     171       566  
3         5426     137       472  
4         9322     199       408  
Meta ads:
                            campaign_id campaign_name        date  reach  \
0  c6429538-73ff-40f3-a67d-dc003e56a08e     Campaña 4  2024-05-28   8756   
1  bd845c3a-539f-4d71-a947-a31135448161     Campaña 8  2024-01-14   5577   
2  9d3f8f33-20b7-4a3a-a16f-6c842ca14f82     Campaña 2  2024-04-22   7781   
3  494ebf92-6259-4277-8295-25a4152cf1bd     Campa

Paso 2: Agregar columna 'canal' a cada dataset

In [36]:
google['canal']= "Google ads"
meta['canal']  = "Meta ads"
email['canal'] = "Email marketing"

Paso 3: Homogeneizar columnas

In [37]:
# Renombrar columnas para consistencia
google.rename(columns={"impressions": "impresiones", "clicks": "clics", "cost_usd": "costo"}, inplace=True)
meta.rename(columns={"reach": "impresiones", "link_clicks": "clics", "cost_usd": "costo"}, inplace=True)
email.rename(columns={"recipients": "impresiones", "opens": "aperturas", "clicks": "clics"}, inplace=True)

# Agregar columna vacía de 'aperturas' donde no aplica
google["aperturas"] = None
meta["aperturas"] = None
email["costo"] = None  # Supondremos que el envío de emails no tiene costo directo

# Asegurar formato de fecha
google["date"] = pd.to_datetime(google["date"])
meta["date"] = pd.to_datetime(meta["date"])
email["date"] = pd.to_datetime(email["date"])


Paso 4: Unir todos los datasets

In [38]:
columnas = ["campaign_id", "campaign_name", "date", "canal", "impresiones", "aperturas", "clics", "costo"]
google = google[columnas]
meta   = meta[columnas]
email  = email[columnas]

#concatenar dataset
df_unificado = pd.concat([google,meta,email], ignore_index=True)

#vista previa
df_unificado.head()

,campaign_id,campaign_name,date,canal,impresiones,aperturas,clics,costo
0,16a0a598-498c-4360-acbd-73fd5de3c96c,Campaña 10,2024-02-06,Google ads,8270,None,485,960
1,095727ce-b041-47b8-b4a3-5bdc81e9fb10,Campaña 10,2024-02-07,Google ads,6390,None,156,171
2,3aa3a3d5-b342-4fe1-b6c4-8e69f0924bef,Campaña 8,2024-02-23,Google ads,6734,None,171,566
3,d05b281b-deb2-4e0d-bd4e-b0fda784ad72,Campaña 6,2024-02-14,Google ads,5426,None,137,472
4,06f8ef35-9939-45bf-bc16-d02d1074940e,Campaña 1,2024-04-20,Google ads,9322,None,199,408


Paso 5: Verificar nulos y tipos

In [ ]:
#Nulos
print(df_unificado.isnull().sum)

#Tipo de datos
print(df_unificado.dtypes)


## Fase 2: Transformación y KPIs

In [43]:
# Evitar división por cero
df_unificado["impresiones"] = df_unificado["impresiones"].replace(0, pd.NA)
df_unificado["clics"] = df_unificado["clics"].replace(0, pd.NA)

# CTR (Tasa de clics)
df_unificado["ctr"] = df_unificado["clics"]/ df_unificado["impresiones"]

#costo por clic 
df_unificado["cpc"] = df_unificado["costo"]/df_unificado["clics"]

Paso 2: Validación de datos sospechosos

Campañas con CTR fuera de rango esperado (>0.5 o <0.005)

In [44]:
ctr_Altos = df_unificado[df_unificado["ctr"] > 0.5]
cpc_Bajos = df_unificado[df_unificado["cpc"] < 0.005]

Costos sin clics (puede indicar error o falta de datos)

In [47]:
costos_sin_clics = df_unificado[(df_unificado["costo"].notnull()) & (df_unificado["clics"].isnull())]

Campañas duplicadas

In [48]:
duplicadas = df_unificado[df_unificado.duplicated(["campaign_id", "date", "canal"], keep=False)]


Paso 3: KPIs agregados por canal

In [58]:
# KPIs por canal
kpis_por_canal = df_unificado.groupby("canal").agg({
    "impresiones": "sum",
    "clics": "sum",
    "costo": "sum"
}).reset_index()

# Calcular KPIs derivados
kpis_por_canal["ctr_promedio"] = kpis_por_canal["clics"] / kpis_por_canal["impresiones"]
kpis_por_canal["cpc_promedio"] = kpis_por_canal["costo"] / kpis_por_canal["clics"]

print("KPIs por canal:")

KPIs por canal:


Paso 4: Top 5 campañas con mejor CTR

In [57]:
top_ctr = df_unificado[df_unificado["ctr"].notnull()].sort_values("ctr", ascending=False).head(5)
print(top_ctr[["campaign_name", "canal", "ctr", "impresiones", "clics"]])


    campaign_name            canal       ctr  impresiones  clics
335    Campaña 10         Meta ads  0.436025          805    351
337     Campaña 6         Meta ads  0.434024          917    398
12      Campaña 2       Google ads  0.416316         1189    495
455     Campaña 2         Meta ads  0.406178          874    355
626     Campaña 5  Email marketing  0.403526         1021    412


Paso 5: Exportar archivos para Power BI

In [59]:
# Datos consolidados
df_unificado.to_csv("df_campañas_limpias.csv", index=False)

# KPIs por canal
kpis_por_canal.to_csv("kpis_campañas.csv", index=False)

# Exportar campañas con errores para auditoría
costos_sin_clics.to_csv("campañas_con_errores.csv", index=False)
